##Importing Libraries and Loading Dataset


In [2]:
import pandas as pd
import numpy as np

In [3]:
!unzip '/content/Fetch_Data.zip'

Archive:  /content/Fetch_Data.zip
   creating: Fetch_Data/
  inflating: Fetch_Data/PRODUCTS_TAKEHOME.csv  
  inflating: Fetch_Data/TRANSACTION_TAKEHOME.csv  
  inflating: Fetch_Data/USER_TAKEHOME.csv  


In [4]:
# Loading Data
products_df = pd.read_csv('/content/Fetch_Data/PRODUCTS_TAKEHOME.csv')
transaction_df = pd.read_csv('/content/Fetch_Data/TRANSACTION_TAKEHOME.csv')
user_df = pd.read_csv('/content/Fetch_Data/USER_TAKEHOME.csv')

##Basic Data Quality Check:

In [5]:
def basic_data_cleaning(df,dfname): #Inputs for this function are the dataframe and it's name.

    #Length of Df before cleaning
    print(f"Length of {dfname} before cleaning:")
    print(len(df))
    print("\n")

    #Counting null values and calculating percentage of null values in each column if present
    null_counts = df.isnull().sum()
    print(f"Null values in {dfname}:")
    print(null_counts)
    print("\n")
    if null_counts.any():  # Check if there are any null values
        total_rows = len(df)
        null_percentages = (null_counts[null_counts > 0] / total_rows) * 100
        print(f"Percentage of missing data in {dfname}:") # Prints Percentage
        print(null_percentages)
        print("\n")

    #Counting and dropping duplicates if Present
    duplicate_rows = df[df.duplicated()]
    num_duplicates = duplicate_rows.shape[0]
    print(f"Number of duplicate rows in {dfname}: {num_duplicates}")
    if num_duplicates > 0:
      df = df.drop_duplicates()
      print(f"Duplicates dropped in {dfname}.")
    else:
      print(f"No Duplicates found in {dfname}.")
    print("\n")

    #Standardizing text formatting by removing white spaces and converting to lower case
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.strip().str.lower()
        print(f"Text formatting standardized in {dfname}.")
    print("\n")

    #Checking datatypes
    print(f"Datatypes in {dfname}:")
    print(df.dtypes)
    print("\n")

    #Length of Df after basic cleaning
    print(f"Length of {dfname} after basic cleaning:")
    print(len(df))
    print("\n")

    return df

##Products

In [6]:
cleaned_products_df = basic_data_cleaning(products_df,'products_df')

Length of products_df before cleaning:
845552


Null values in products_df:
CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64


Percentage of missing data in products_df:
CATEGORY_1       0.013128
CATEGORY_2       0.168411
CATEGORY_3       7.162895
CATEGORY_4      92.021898
MANUFACTURER    26.784160
BRAND           26.783923
BARCODE          0.476020
dtype: float64


Number of duplicate rows in products_df: 215
Duplicates dropped in products_df.




<ipython-input-5-91174e99805f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip().str.lower()


Text formatting standardized in products_df.
Text formatting standardized in products_df.
Text formatting standardized in products_df.
Text formatting standardized in products_df.
Text formatting standardized in products_df.
Text formatting standardized in products_df.


Datatypes in products_df:
CATEGORY_1       object
CATEGORY_2       object
CATEGORY_3       object
CATEGORY_4       object
MANUFACTURER     object
BRAND            object
BARCODE         float64
dtype: object


Length of products_df after basic cleaning:
845337




**Initial Impressions:**

We can see that there is a lot of null values in this dataset.
For eg: 92% of rows in Category_4 are missing.

There are 215 duplicate rows in this dataset and they have been removed.

Numeric column "Barcode" is in object datatype. It has to be changed to INT.

In [7]:
cleaned_products_df[cleaned_products_df['BARCODE'] == ''] # No empty strings are present in this column

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE


In [8]:
unique_count = cleaned_products_df['BARCODE'].nunique()
print(unique_count)

841342


In [9]:
# Example of Duplicate Barcode in Products Df
cleaned_products_df[cleaned_products_df['BARCODE'] == 75053055]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
114131,snacks,candy,chocolate candy,NaN,grupo nacional de chocolates sa,nutresa,75053055.0
181902,snacks,candy,confection candy,NaN,grupo nacional de chocolates sa,nutresa,75053055.0


Out of 845337 values, 841342 values in the Barcode column are unique, 4025 values are null and there are no empty strings. This can be used as the primary key in SQL for the products table after removing the duplicates and null values.


In [10]:
#Removing null and duplicates
cleaned_products_df = cleaned_products_df.dropna(subset=['BARCODE']) # Dropping Null Values
cleaned_products_df = cleaned_products_df.drop_duplicates(subset=['BARCODE'])

In [11]:
cleaned_products_df['BARCODE'] = cleaned_products_df['BARCODE'].astype('int')

<ipython-input-11-e87f789ed910>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_products_df['BARCODE'] = cleaned_products_df['BARCODE'].astype('int')


In [12]:
len(cleaned_products_df) # New length of Df after removing null and duplicates

841342

In [13]:
cleaned_products_df.head()

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,health & wellness,sexual health,conductivity gels & lotions,NaN,NaN,NaN,796494407820
1,snacks,puffed snacks,cheese curls & puffs,NaN,NaN,NaN,23278011028
2,health & wellness,hair care,hair care accessories,NaN,placeholder manufacturer,elecsop,461817824225
3,health & wellness,oral care,toothpaste,NaN,colgate-palmolive,colgate,35000466815
4,health & wellness,medicines & treatments,essential oils,NaN,maple holistics and honeydew products intercha...,maple holistics,806810850459


In [14]:
cleaned_products_df[cleaned_products_df['MANUFACTURER'] == 'placeholder manufacturer'].shape[0]

86895

There a 226474 null values in the manufacturer column but it also has a placeholder string "placeholder manufacturer" 86895 times.

How should we handle these placeholder strings?

Should they be considered as null values?

In [15]:
cleaned_products_df.to_csv('cleaned_products.csv', index=False) #Exporting df to csv

##Users

In [16]:
cleaned_user_df = basic_data_cleaning(user_df,'user_df')

Length of user_df before cleaning:
100000


Null values in user_df:
ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64


Percentage of missing data in user_df:
BIRTH_DATE     3.675
STATE          4.812
LANGUAGE      30.508
GENDER         5.892
dtype: float64


Number of duplicate rows in user_df: 0
No Duplicates found in user_df.


Text formatting standardized in user_df.
Text formatting standardized in user_df.
Text formatting standardized in user_df.
Text formatting standardized in user_df.
Text formatting standardized in user_df.
Text formatting standardized in user_df.


Datatypes in user_df:
ID              object
CREATED_DATE    object
BIRTH_DATE      object
STATE           object
LANGUAGE        object
GENDER          object
dtype: object


Length of user_df after basic cleaning:
100000




**Initial Impressions:**

We can see that there is a lot of null values in this dataset.
For eg: 30% of rows in Language are missing.

There are no duplicate rows in this dataset.

Datetime columns "CREATED_DATE" and "BIRTH_DATE" are in object datatype. It has to be changed to datetime.

In [17]:
cleaned_user_df[cleaned_user_df['ID'] == ''] # No empty strings are present in this column

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER


In [18]:
unique_count = cleaned_user_df['ID'].nunique()
print(unique_count)

100000


All the values in the ID column are unique, no null values and there are no empty strings. This can be used as the primary key in SQL for the Users table.

In [19]:
cleaned_user_df['CREATED_DATE'] = pd.to_datetime(cleaned_user_df['CREATED_DATE'])
cleaned_user_df['BIRTH_DATE'] = pd.to_datetime(cleaned_user_df['BIRTH_DATE'])

In [20]:
user_df.GENDER.value_counts()

,count
GENDER,
female,64240
male,25829
transgender,1772
prefer_not_to_say,1350
non_binary,473
unknown,196
not_listed,180
non-binary,34
not_specified,28


There is inconsistency in the Gender terms and it has to be normalized

In [21]:
#Normalizing Gender terms
cleaned_user_df['GENDER'] = cleaned_user_df['GENDER'].str.replace('_', ' ')
cleaned_user_df['GENDER'] = cleaned_user_df['GENDER'].str.replace('-', ' ')
cleaned_user_df['GENDER'] = cleaned_user_df['GENDER'].str.replace('not Specified', 'not Listed')
cleaned_user_df['GENDER'] = cleaned_user_df['GENDER'].str.replace('my gender isn\'t listed', 'not listed')

In [22]:
cleaned_user_df['GENDER'].value_counts()  #Count after Gender Normalization

,count
GENDER,
female,64240
male,25829
transgender,1772
prefer not to say,1351
non binary,507
unknown,196
not listed,185
not specified,28


In [23]:
# To check if there are invalid entries in the state column
print(cleaned_user_df['STATE'].value_counts())
print("Number of Unique states in the State columns: ",cleaned_user_df['STATE'].nunique())

STATE
tx    9028
fl    8921
ca    8589
ny    5703
il    3794
ga    3718
pa    3644
oh    3519
nc    3399
nj    2899
mi    2582
va    2400
in    2213
tn    2165
al    1792
sc    1773
ky    1731
az    1728
mo    1652
md    1636
ma    1523
wi    1389
pr    1361
la    1290
wa    1234
co    1139
mn    1125
ct    1110
ok    1100
ar    1011
ms     960
ia     933
nv     837
ks     835
or     700
wv     674
ut     593
de     492
ne     487
nm     452
me     368
nh     359
hi     350
id     350
ak     297
ri     284
dc     241
sd     205
nd     185
mt     169
wy     138
vt     111
Name: count, dtype: int64
Number of Unique states in the State columns:  52


PR = Puerto Rico and DC - District of Columbia are represented as states in this dataset.

I’m raising a flag for these as they aren't valid U.S. states. We should decide how to handle them in our analysis moving forward.

In [24]:
#Checking Inconsistency in Birth_date column
#Calculating Age of User
current_date = pd.Timestamp.now()
cleaned_user_df['age'] = (current_date - cleaned_user_df['BIRTH_DATE']).dt.days // 365

# Checking Invalid ages (ages less than 5 or ages greater 100)
invalid_ages = cleaned_user_df[(cleaned_user_df['age'] < 5) | (cleaned_user_df['age'] > 100)]
print(invalid_ages[['ID','age']])
print(len(invalid_ages))

                             ID    age
3656   62be5974baa38d1a1f6b6725  121.0
5190   5f20a5a2b06a3214c6db4811  101.0
5878   60ac6acc79ed9200a6ebc24a  123.0
6499   60a6e4af3369535cb6c4c89d  123.0
7723   624501294eadbb0ccddf2cc6  101.0
...                         ...    ...
96783  5ee14ae9e7bf80140949615c  117.0
98338  6418e8d1197cb65cc272db79  118.0
98639  61a1127207acef7276e6158f  123.0
98847  60bc0f11a3100c10461d86fa  123.0
98873  5ca8bf5c5f04d512fc5d3dc6  106.0

[75 rows x 2 columns]
75


The Dataset has 75 invalid ages and it has to be removed.

In [25]:
cleaned_user_df = cleaned_user_df[~cleaned_user_df['ID'].isin(invalid_ages['ID'])] #Removing invalid ages

In [26]:
cleaned_user_df.drop(columns=['age'], inplace=True) # Dropping Age column as it was created for quality check

<ipython-input-26-002728142e73>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_user_df.drop(columns=['age'], inplace=True) # Dropping Age column as it was created for quality check


In [27]:
len(cleaned_user_df)

99925

In [28]:
cleaned_user_df['LANGUAGE'].value_counts()

,count
LANGUAGE,
en,63381
es-419,6086


There are only 2 languages in the dataset. English (EN) and Spanish appropriate for the Latin America and Caribbean region (ES-419)

In [29]:
cleaned_user_df.to_csv('cleaned_user.csv', index=False)  #Exporting df to csv

## Transaction

In [30]:
cleaned_transaction_df = basic_data_cleaning(transaction_df,'transaction_df')

Length of transaction_df before cleaning:
50000


Null values in transaction_df:
RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64


Percentage of missing data in transaction_df:
BARCODE    11.524
dtype: float64


Number of duplicate rows in transaction_df: 171
Duplicates dropped in transaction_df.


Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.
Text formatting standardized in transaction_df.


Datatypes in transaction_df:
RECEIPT_ID         object
PURCHASE_DATE      object
SCAN_DATE          object
STORE_NAME         object
USER_ID            object
BARCODE           float64
FINAL_QUANTITY     object
FI

<ipython-input-5-91174e99805f>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.strip().str.lower()


**Initial Impressions:**

We can see that there are null values only in one column in this dataset.

The barcode column has 5762 null values which is 11.52% and they can be removed for the purpose of this analysis.

There are 171 duplicate rows in this dataset and they have been removed.

Barcode column has to be changed to INT Datatype.

Datetime columns "PURCHASE_DATE" and "SCAN_DATE" are in object datatype. It has to be changed to datetime.

Numeric columns "FINAL_QUANTITY" and "FINAL_SALE" are in object datatype. It has to be changed to Numeric.

In [31]:
cleaned_transaction_df = cleaned_transaction_df.dropna(subset=['BARCODE']) #removing null values
cleaned_transaction_df['BARCODE'] = cleaned_transaction_df['BARCODE'].astype('int') #Changing datatype to int

In [32]:
cleaned_transaction_df['RECEIPT_ID'].nunique() # number of unique receipt IDs in the dataset

21639

In [33]:
cleaned_transaction_df['BARCODE'].nunique() # number of unique barcodes in the dataset

11027

In [34]:
cleaned_transaction_df[cleaned_transaction_df['RECEIPT_ID'] == '98d68d5d-71f1-4528-a83d-cdf6d308c79b']

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
14978,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,28400090896,1.00,
14979,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,28400091510,zero,0.49
14980,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,652729710588,1.00,
30380,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,28400091510,1.00,0.49
31616,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,652729710588,1.00,1.48
44637,98d68d5d-71f1-4528-a83d-cdf6d308c79b,2024-08-12,2024-08-12 12:40:23.184 z,walmart,64486dd4347839fdfa9a28bb,28400090896,1.00,0.49


In [35]:
cleaned_transaction_df[cleaned_transaction_df['BARCODE'] == 78742223759]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
347,03a45f0c-a6b0-4f3a-91f4-d7af829d1681,2024-07-04,2024-07-04 13:13:13.600 z,walmart,5b61c5face7e6314a612495a,78742223759,zero,1.46
957,0991b11b-bcde-41ff-b794-b7f3151984e5,2024-07-04,2024-07-05 12:49:21.166 z,walmart,665bac1de04f743a096cbfb0,78742223759,zero,2.92
1055,0a8bebfc-2128-41d5-8026-80a9ac90f22b,2024-07-11,2024-07-26 00:22:23.246 z,walmart,638141d4d6f2a49c49981da6,78742223759,zero,1.46
1123,0b5644e3-f3c4-45c4-b20c-7ab56964a2fb,2024-07-02,2024-07-02 18:23:02.927 z,walmart,64305e1e44f22b2ce4309794,78742223759,zero,1.53
1959,14247c1d-d2b7-4987-ba32-dcdc563b6042,2024-07-11,2024-07-31 15:10:08.242 z,walmart,61a198005889930cbd6615f3,78742223759,zero,1.46
...,...,...,...,...,...,...,...,...
45864,845f7749-42eb-48ae-9dfc-fd1921660206,2024-07-27,2024-08-11 09:06:08.831 z,walmart,612d19c002c1042722232287,78742223759,1.00,1.46
45867,7b7976b1-eea0-40a2-a5e5-2a08a6b98236,2024-07-11,2024-07-15 15:53:27.767 z,walmart,62fa3e5d92710024ad017e56,78742223759,5.00,7.30
45868,51ca4a2c-a14b-4f48-9875-98f0318cd1b2,2024-07-03,2024-07-04 13:16:07.278 z,walmart,606688e5d9ffaa2d900e5d7a,78742223759,3.00,4.38
45869,f274d2d0-4c9a-4217-bdc8-f06c58e9c792,2024-07-08,2024-07-08 16:10:38.007 z,walmart,63adc1c301cb7c888c705694,78742223759,1.00,1.46


Out of 49829 values, there are only 21639 unique values in the receipt ID column and 11027 unique barcodes. I believe that is because a receipt IDS are not unique because multiple products would be listed in it and each product would be in a separate row. In the case of barcode, different receipts can have the same products.

Hence, Receipt_ID cannot be used as the primary key for this table. A possible solution is to keep another column with sequential product numbering for each receipt and use that column with receipt id to create a composite key.

In [36]:
cleaned_transaction_df.FINAL_QUANTITY.value_counts()

,count
FINAL_QUANTITY,
1.00,31440
zero,11067
2.00,1162
3.00,157
4.00,121
...,...
1.28,1
2.57,1
1.07,1


It can be seen that 0 is represented as a string ("zero") in this column and it has to be changed.

In [37]:
cleaned_transaction_df['FINAL_QUANTITY'] = cleaned_transaction_df['FINAL_QUANTITY'].replace('zero', 0.0) #Replacing "zero" to 0

In [38]:
cleaned_transaction_df['FINAL_QUANTITY'] = pd.to_numeric(cleaned_transaction_df['FINAL_QUANTITY'], errors='coerce') #changing object datatype in numeric

In [39]:
# Remove rows where FINAL_QUANTITY is less than or equal to 0 or is NaN
cleaned_transaction_df = cleaned_transaction_df[cleaned_transaction_df['FINAL_QUANTITY'].notna() & (cleaned_transaction_df['FINAL_QUANTITY'] > 0)]

The values in Final Quantity cannot be 0 or less than 0 as the user must have purchased a product for it to be printed on a receipt

In [40]:
cleaned_transaction_df['FINAL_SALE'].value_counts()

,count
FINAL_SALE,
,11031
1.25,739
1.00,451
2.00,346
3.98,335
...,...
7.45,1
8.64,1
17.64,1


It can be seen that null values are represented by an empty string and it should be changed.

In [41]:
cleaned_transaction_df['FINAL_SALE'] = cleaned_transaction_df['FINAL_SALE'].replace('', np.nan) #Replacing empty string to null

<ipython-input-41-2b4e29950e1d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_transaction_df['FINAL_SALE'] = cleaned_transaction_df['FINAL_SALE'].replace('', np.nan) #Replacing empty string to null


In [42]:
cleaned_transaction_df['FINAL_SALE'] = pd.to_numeric(cleaned_transaction_df['FINAL_SALE'], errors='coerce')  #changing object datatype in numeric

<ipython-input-42-d93a57f4b7a8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_transaction_df['FINAL_SALE'] = pd.to_numeric(cleaned_transaction_df['FINAL_SALE'], errors='coerce')  #changing object datatype in numeric


In [43]:
# Remove rows where FINAL_SALE is less than or equal to 0 or is NaN
cleaned_transaction_df = cleaned_transaction_df[cleaned_transaction_df['FINAL_SALE'].notna() & (cleaned_transaction_df['FINAL_SALE'] > 0)]

The values in Final Sale cannot be 0 or less than 0 as the user must have purchased a product for it to be printed on a receipt

In [44]:
cleaned_transaction_df['PURCHASE_DATE'] = pd.to_datetime(cleaned_transaction_df['PURCHASE_DATE'])
cleaned_transaction_df['SCAN_DATE'] = pd.to_datetime(cleaned_transaction_df['SCAN_DATE'])

In [45]:
#Checking Consistency of the datetime columns
# Scan Dates cannot be before Purchase Dates
print("Inconsistent dates: ")
inconsistent_dates = cleaned_transaction_df[cleaned_transaction_df['PURCHASE_DATE'] > cleaned_transaction_df['SCAN_DATE']]
print(inconsistent_dates.shape[0])

Inconsistent dates: 
47


In [46]:
#Sample Inconsistent dates
print(inconsistent_dates[['RECEIPT_ID','PURCHASE_DATE','SCAN_DATE']].head().to_string(index=False))

                          RECEIPT_ID PURCHASE_DATE               SCAN_DATE
5c10e8df-9a9b-4ce0-bfed-51ab7adf2eb5    2024-07-06 2024-07-05 13:18:03.207
05023b3d-5f83-47a7-a17c-8e8521d0bc94    2024-09-08 2024-09-07 22:22:29.903
bfa85c94-a644-4f81-8a83-90ade603cf31    2024-06-15 2024-06-14 21:26:00.636
878fe0e9-42e5-4de2-b9d1-da82ecf25db1    2024-06-29 2024-06-28 20:04:50.390
801eeda5-e4fe-41e1-97c2-03026765cc47    2024-08-31 2024-08-30 19:27:02.783


In the First row, it can be seen that the purchase date "2024-07-21" is after the scan date "2024-07-20". This is invalid because purchase date cannot be after scan date. So, this has to be removed.

In [47]:
cleaned_transaction_df = cleaned_transaction_df[~(cleaned_transaction_df['PURCHASE_DATE'] > cleaned_transaction_df['SCAN_DATE'])]

In [48]:
len(cleaned_transaction_df)

21835

The 47 rows that had inconsistent dates have been removed

In [49]:
cleaned_transaction_df[cleaned_transaction_df['USER_ID'] == '63b73a7f3d310dceeabd4758']

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539,walmart,63b73a7f3d310dceeabd4758,15300014978,1.0,1.54


In [50]:
cleaned_user_df[cleaned_user_df['ID'] == '63b73a7f3d310dceeabd4758']

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER


There are IDs in the Transaction table that are not present in the user table. This is logically not possible as a user must create an account before scanning receipts using the app.

In [51]:
cleaned_transaction_df.USER_ID.nunique() # Before Filtering based on IDs in user table

15964

In [52]:
user_transaction_df = cleaned_transaction_df[cleaned_transaction_df['USER_ID'].isin(cleaned_user_df['ID'])]

In [53]:
len(cleaned_transaction_df)

21835

In [54]:
user_transaction_df.USER_ID.nunique() # After Filtering based on IDs in user table

85

There are only 85 unique user IDs in the transaction table that correspond to data in the User table.

This is a big data integrity issue as the transaction table contains 15,879 Unique IDs that are not present in the User table.

The dataset must be validated as this will impact any analysis made.

In [55]:
cleaned_products_df[cleaned_products_df['BARCODE'] == 24000393429]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE


In [56]:
cleaned_transaction_df[cleaned_transaction_df['BARCODE'] == 24000393429]

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
25004,21feab39-49f2-42e9-ae69-10371e2fc0a9,2024-08-23,2024-08-27 10:45:00.125,target,61a58ac49c135b462ccddd1c,24000393429,1.0,2.59
36117,027f85a6-65aa-4dfd-a323-e10093953b66,2024-07-16,2024-07-16 15:31:40.795,walmart,5e718f029559aa138177814c,24000393429,1.0,2.63
49658,05f0daef-b9ee-4e8b-9569-4387bce6886f,2024-07-17,2024-07-31 13:07:46.658,walmart,5d6a7cd10d1bff431660c97f,24000393429,1.0,2.50


There are products in the Transaction table that are not present in the products table. Although it is logically possible that some products might not be present in the dataset, they should be added in the future.

In [57]:
product_user_transaction_df = user_transaction_df[user_transaction_df['BARCODE'].isin(cleaned_products_df['BARCODE'])]

This Dataframe contains rows from transaction table that have corresponding data in Users table(ID) and Products Table(Barcode)

In [58]:
product_user_transaction_df

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
25241,86f6d365-1ddd-4f8b-bb6a-b924d4a54905,2024-08-03,2024-08-03 17:19:09.102,walmart,63c8294d39c79dcbdd5c1e4f,44000031114,1.0,3.88
25459,c61751a9-9995-4eb3-8e39-d5fd7b08774f,2024-08-11,2024-08-13 13:00:38.933,save mart supermarkets,5bb79c65b0a16836db35bbf1,311111423818,1.0,5.79
25489,9a6e4c08-b07b-4bad-ad44-330d2da1b67d,2024-06-17,2024-06-17 11:20:43.843,walmart,62815e99c907cf3f47d8ee35,810021200231,1.0,1.97
25712,a713af20-8268-4094-8a41-3521e818bd15,2024-06-15,2024-06-15 12:54:12.582,target,61ca1f591a3acd27c1b18f0b,28400517942,1.0,5.19
25874,a7e6adf6-3dac-497f-90f1-7f371c639a1f,2024-07-22,2024-07-22 09:49:41.406,target,5b441360be53340f289b0795,2700717433990,1.0,7.99
...,...,...,...,...,...,...,...,...
47409,7297a22f-4a0a-4377-9eae-c8fb3e35ea4c,2024-08-25,2024-08-25 15:42:48.533,walmart,6615dab878ee6750bbc350ea,817719021673,1.0,6.98
47670,ad069462-7805-4c2b-a75f-283e5f63d851,2024-08-05,2024-08-13 07:53:18.199,walmart,62c09104baa38d1a1f6c260e,28400324434,1.0,2.32
48172,d80f7f61-d98f-46d7-be8a-0969971e2b7c,2024-08-16,2024-08-17 08:55:14.932,hy-vee,5e6d0beb01ecdd13986852e8,30772112526,1.0,8.97
48691,47c724c5-680f-450d-a617-e36514fff06e,2024-07-18,2024-07-19 18:36:31.512,h-e-b,64ce5d823cb069b5eac9b700,41220078400,1.0,2.97


In [59]:
len(cleaned_transaction_df)

21835

In [60]:
len(product_user_transaction_df)

72

The product_user_transaction_df only has 72 rows because this df contains only the rows from transaction table that have corresponding data in both the products and user tables.

In [61]:
cleaned_transaction_df.to_csv('cleaned_transaction.csv', index=False) #Exporting df to csv